In [36]:
 #PWM_DIFF_CALCULATOR.IPYNB - calculating a bound on the pulse interval to cap deviation from the mean in PWM control

 # source for diffusion coeff: https://journals.asm.org/doi/10.1128/jb.185.5.1485-1491.2003#T1

# PACKAGE IMPORTS
import numpy as np
import scipy
import jax
import jax.numpy as jnp
import jaxopt
import functools
import diffrax as diffrax
from diffrax import diffeqsolve, ODETerm, SaveAt, PIDController, SteadyStateEvent

# plotting and data handling
import pandas as pd
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, palettes as bkpalettes, io as bkio
from bokeh.colors import RGB as bkRGB
from contourpy import contour_generator as cgen
import matplotlib as mpltlb

# miscellaneous
import time

# PACKAGE SETUP
# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

Loading BokehJS ...

In [37]:
# CALCULATOR DEFINITION

def T_alpha_calc(alpha,  # maximum deviation from assumed inducer level, % of pulse concentration u_max
            l,  # length of tubing between the valve and the microfluidic chip, cm
            V,  # volume of 1 cm of tubing, uL
            nu,  # flow rate of the medium through the tubing, uL/min
            D,  # diffusion coefficient of the inducer, cm^2/min
            ):
    # get the Lambda value
    Lambda = 0.5*np.pi*(1+alpha/100)

    # get the r_alpha
    r_alpha = (2*Lambda+1-np.sqrt(4*Lambda+1))/(2*Lambda)

    # get the T_alpha
    T_alpha = np.sqrt(- ((np.pi**2) * l * (V**3) * D) /
                      ((nu**3) * np.log(r_alpha)))

    # return the calculated T_alpha
    return T_alpha

In [38]:
# DEFINE PARAMETERS

# get the volume of 1 cm of tubing, uL
internal_diameter_in = 1/32  # internal diameter of the tubing in inches
internal_radius_m = internal_diameter_in * 0.0254 / 2  # internal radius in metres
volume_per_cm_m3 = np.pi * (internal_radius_m**2) /100  # volume of 1 cm of tubing in m^3
V = volume_per_cm_m3 * 1e9  # volume of 1 cm of tubing in uL
print(f"Volume of 1 cm of tubing: {V} uL")

# set the flow rate of the medium through the tubing, uL/min
nu = 20  # uL/min - common cruise flow rate
print(f"Flow rate of the medium through the tubing: {nu} uL/min")

# set the diffusion coefficient of the inducer, cm^2/min
D_cm2_s = 8.8e-6  # cm^2/s - diffusion coefficient of inducer in water at 37C
D = D_cm2_s * 60  # diffusion coefficient in cm^2/min
print(f"Diffusion coefficient of the inducer: {D} cm^2/min")


Volume of 1 cm of tubing: 4.948315225561501 uL
Flow rate of the medium through the tubing: 20 uL/min
Diffusion coefficient of the inducer: 0.000528 cm^2/min


In [41]:
# CALCULATE T_ALPHA for given l values

# set the maximum deviation from the assumed inducer level, % of pulse concentration u_max
alpha = 5

# get the range of l values to consider
l_values = np.linspace(10, 100, 9)
T_alphas = [T_alpha_calc(alpha, l, V, nu, D) for l in l_values]
control_delays = V*l_values/nu


In [44]:
# PLOT
# initialise
fig = bkplot.figure(
    frame_width=240,
    frame_height=180,
    y_range=(0, 1.25 * np.max(T_alphas)),
    x_axis_label="Tubing length, cm",
    y_axis_label="T_5%, min",
    tools="box_zoom,pan,hover,reset,save"
)
# set svg backend
fig.output_backend = "svg"

# settings for the main y-axis (for p_cat)
fig.yaxis.axis_line_color=bkRGB(222, 49, 99)
fig.yaxis.major_tick_line_color=bkRGB(222, 49, 99)
fig.yaxis.minor_tick_line_color=bkRGB(222, 49, 99)

# plot the cat protein concentrations
fig.line(l_values,T_alphas, line_width=2, line_color=bkRGB(222, 49, 99), legend_label="T_5%")

# create an extra  y range for plotting integrase protein concentrations
fig.extra_y_ranges = {"t~": bkmodels.Range1d(start=0, end=1.25 * np.max(control_delays))}
fig.add_layout(bkmodels.LinearAxis(y_range_name="t~",
                                     axis_label="Control delay, min",
                                     axis_line_color=bkRGB(255, 103, 0),
                                     major_tick_line_color=bkRGB(255, 103, 0),
                                     minor_tick_line_color=bkRGB(255, 103, 0)),
                 'right')  # add the alternative axis label to the figure

# plot the integrase protein concentrations
fig.line(l_values,control_delays, line_width=2, line_color=bkRGB(255, 103, 0), y_range_name="t~", legend_label="Control delay")

# add legend
fig.legend.location = "right"
fig.legend.label_text_font_size="8pt"

# set fonts
fig.xaxis.axis_label_text_font_size = "8pt"
fig.xaxis.major_label_text_font_size = "8pt"
fig.yaxis.axis_label_text_font_size = "8pt"
fig.yaxis.major_label_text_font_size = "8pt"

# show plots
bkplot.show(fig)